# Set up

In [ ]:
# Import libraries
import os
import re
import sqlite3
from datetime import datetime

import numpy as np
import pandas as pd


# Function to extract score from options based on values in the response
def val_score_mapping(s1, s2):
    split_options = s2.strip().split("),")
    split_response = s1.strip().split(": ")[1].split(",")
    scores = {}

    for i in split_options:
        if len(i) > 0:
            val_num = i.split("(score")[0].split(": ")[1].strip()
            score_num = i.split("(score")[1].split(": ")[1].strip()
            scores[val_num] = score_num

    response_score_mapping = {
        split_response[i].strip(): scores[split_response[i].strip()]
        for i in range(len(split_response))
    }
    list_response_score_mapping = list(response_score_mapping.values())
    str_response_score_mapping = ", ".join(
        str(value) for value in list_response_score_mapping
    )
    return str_response_score_mapping.replace(")", "")


# Function to cleanup and split time range in the response
def clean_time_range(df, column_name):
    cleaned = []
    for i in range(len(df[column_name])):
        if pd.notna(df[column_name][i]) and str(df[column_name][i]).startswith(
            "time_range"
        ):
            t = re.sub(r"[a-zA-Z\s+(\)_:]", "", df[column_name][i])
            t = t.replace(",", ":")
            if re.search(r"^[0-9]:", t):  # 9,30/12,30
                ttemp = "0" + t
            elif re.search(r":[0-9]$", t):  # 12,5/12,30
                ttemp = t.replace(":", ":0")
            else:
                ttemp = t
            # tpos = datetime.strptime(str(ttemp), '%H:%M')
            # thm = tpos.strftime('%H:%M')
            thm = ttemp
        else:
            ttemp = df[column_name][i]
            thm = ttemp
        cleaned.append(thm)
    return cleaned

#### Read in data

In [ ]:
# setup path variable and get list of report.csv files

input_path = os.path.expanduser("~/NIMH EMA Data v2/Input Files/")  # noqa: PTH111
all_files = os.listdir(os.path.join(input_path, "EMA_applet_data"))
files = [file for file in all_files if file.startswith("report")]
input_files = os.listdir(input_path)
output_path = os.path.expanduser("~/NIMH EMA Data v2/Output Files/")  # noqa: PTH111

# Read all the report.csv files
report_all = []
for i in range(len(files)):
    temp_df = pd.read_csv(
        os.path.join(input_path, "EMA_applet_data", files[i]), encoding="ISO-8859-1"
    )
    report_all.append(temp_df)

# Concat report.csv to one file and read other input files
dat_full = pd.concat(report_all, ignore_index=True)
flow = pd.read_csv(os.path.join(input_path, "flow-items.csv"))
history = pd.read_csv(os.path.join(input_path, "user-flow-schedule.csv"))
if "user-activity-schedule.csv" in input_files:
    act_history_exist = 1
    act_history = pd.read_csv(os.path.join(input_path, "user-activity-schedule.csv"))
else:
    act_history_exist = 0
dat_full = dat_full.applymap(str)

# Write out the concatenated report.csv file
dat_full.to_csv(os.path.join(output_path, "report_all.csv"), index=False)

# Data Cleaning

In [ ]:
# rename id column as it contains special characters
dat_full.rename(columns={dat_full.columns[0]: "id"}, inplace=True)

# Add timezone_offset
dat_full["offset"] = np.where(dat_full["timezone_offset"] == "nan", 0, 1)

dat_full["activity_start_time_offsetADD"] = np.where(
    dat_full["timezone_offset"] != "nan",
    pd.to_numeric(dat_full["activity_start_time"], errors="coerce")
    + (pd.to_numeric(dat_full["timezone_offset"], errors="coerce") * 60 * 1000),
    pd.to_numeric(dat_full["activity_start_time"], errors="coerce"),
)
dat_full["activity_end_time_offsetADD"] = np.where(
    dat_full["timezone_offset"] != "nan",
    pd.to_numeric(dat_full["activity_end_time"], errors="coerce")
    + (pd.to_numeric(dat_full["timezone_offset"], errors="coerce") * 60 * 1000),
    pd.to_numeric(dat_full["activity_end_time"], errors="coerce"),
)
dat_full["activity_scheduled_time_offsetADD"] = np.where(
    dat_full["timezone_offset"] != "nan",
    pd.to_numeric(dat_full["activity_scheduled_time"], errors="coerce")
    + (pd.to_numeric(dat_full["timezone_offset"], errors="coerce") * 60 * 1000),
    pd.to_numeric(dat_full["activity_scheduled_time"], errors="coerce"),
)

dat_full["activity_start_time_offsetADD"] = pd.to_numeric(
    dat_full["activity_start_time_offsetADD"], downcast="integer"
)
dat_full["activity_end_time_offsetADD"] = pd.to_numeric(
    dat_full["activity_end_time_offsetADD"], downcast="integer"
)
dat_full["activity_scheduled_time_offsetADD"] = pd.to_numeric(
    dat_full["activity_scheduled_time_offsetADD"], downcast="integer"
)

### R code translation

In [ ]:
# starting from here is the translation of R code
dat_full["start_Time"] = dat_full["activity_start_time_offsetADD"]
dat_full["end_Time"] = dat_full["activity_end_time_offsetADD"]
dat_full["schedule_Time"] = dat_full["activity_scheduled_time_offsetADD"]

# Cleanup similar to R code
dat_processed = (
    dat_full.groupby(
        ["secret_user_id", "activity_flow_id", "activity_scheduled_time"],
        group_keys=True,
    )
    .apply(
        lambda x: x.assign(
            start_Time=x["start_Time"].min(), end_Time=x["end_Time"].max()
        )
    )
    .reset_index(drop=True)
)

# added extra columns than what exists in R cleanup code
dat_subset = dat_processed[
    [
        "id",
        "activity_scheduled_time",
        "secret_user_id",
        "userId",
        "activity_id",
        "activity_name",
        "activity_flow_id",
        "activity_flow_name",
        "item",
        "response",
        "options",
        "event_id",
        "start_Time",
        "end_Time",
        "schedule_Time",
        "version",
        "activity_start_time",
        "offset",
    ]
]

# Creating additional column to add scores in the next steps
dat_subset = dat_subset.copy()
dat_subset["response_scores"] = None

# Cleanup similar to R code
for i in range(len(dat_subset["response"])):
    if re.search(r"score: ", dat_subset["options"][i]):
        s = val_score_mapping(dat_subset["response"][i], dat_subset["options"][i])
        dat_subset.loc[i, "response_scores"] = s
    if re.search(r"value", dat_subset["response"][i]):
        r = dat_subset["response"][i].replace("value: ", "")
        dat_subset.loc[i, "response"] = r
    elif re.search(r"time:", dat_subset["response"][i]):
        if re.search(r"hr [0-9],", dat_subset["response"][i]):
            egapp = dat_subset["response"][i].replace("time: hr ", "0")
            if re.search(r", min [0-9]$", egapp):
                egtemp = egapp.replace(", min ", ":0")
            elif re.search(r", min [0-9][0-9]$", egapp):
                egtemp = egapp.replace(", min ", ":")
            egpos = datetime.strptime(egtemp, "%H:%M")
            egpos2 = egpos.strftime("%H:%M")
            dat_subset.loc[i, "response"] = egpos2
        elif re.search(r"hr [0-9][0-9],", dat_subset["response"][i]):
            egapp = dat_subset["response"][i].replace("time: hr ", "")
            if re.search(r", min [0-9]$", egapp):
                egtemp = egapp.replace(", min ", ":0")
            elif re.search(r", min [0-9][0-9]$", egapp):
                egtemp = egapp.replace(", min ", ":")
            egpos = datetime.strptime(egtemp, "%H:%M")
            egpos2 = egpos.strftime("%H:%M")
            dat_subset.loc[i, "response"] = egpos2
    elif re.search(r"geo:", dat_subset["response"][i]):
        g = dat_subset["response"][i].replace("geo: ", "")
        dat_subset.loc[i, "response"] = g

# Combining scores and other formats of responses into one column
dat_subset["response2"] = np.where(
    dat_subset["response_scores"].isna(),
    dat_subset["response"],
    dat_subset["response_scores"],
)

# Sorting and Selecting required columns
dat_subset = dat_subset.sort_values(
    by=[
        "secret_user_id",
        "activity_flow_id",
        "activity_id",
        "schedule_Time",
        "activity_start_time",
    ]
)

### Additional Cleaning

In [ ]:
# Creating a new binary to indicate whether responses were from activity or flow
dat_subset["is_activity"] = np.where(dat_subset["activity_flow_id"] == "nan", "Y", "N")

# Creating a new column such that:
#   activity_flow_id if item is from flow
#   activity_id if them is from activity
dat_subset["activity_flow"] = np.where(
    dat_subset["activity_flow_id"] == "nan",
    (dat_subset["activity_id"] + "|" + dat_subset["activity_name"]),
    dat_subset["activity_flow_id"],
)
dat_subset = dat_subset[
    [
        "userId",
        "secret_user_id",
        "activity_flow_id",
        "activity_id",
        "activity_flow",
        "activity_flow_name",
        "is_activity",
        "offset",
        "item",
        "response",
        "response_scores",
        "response2",
        "options",
        "start_Time",
        "end_Time",
        "schedule_Time",
        "activity_scheduled_time",
        "version",
        "id",
        "event_id",
    ]
]

dat_subset = dat_subset.rename(columns={"event_id": "event_id_report"})

# Making sure there are no NAs
dat_subset["schedule_Time"] = np.where(
    dat_subset["schedule_Time"].isna(), "NO SCHEDULE", dat_subset["schedule_Time"]
)

### Widening Data

In [ ]:
# Adding answer_ids to help with debugging in the final output
answers = (
    dat_subset.groupby(
        [
            "userId",
            "secret_user_id",
            "activity_flow",
            "activity_flow_name",
            "event_id_report",
            "is_activity",
            "start_Time",
            "end_Time",
            "schedule_Time",
            "offset",
            "version",
        ]
    )["id"]
    .apply(lambda x: "|".join(x.astype(str)))
    .reset_index()
)

# Widening data
dat_wide = pd.pivot_table(
    dat_subset,
    index=[
        "userId",
        "secret_user_id",
        "activity_flow",
        "activity_flow_name",
        "event_id_report",
        "is_activity",
        "start_Time",
        "end_Time",
        "schedule_Time",
        "offset",
        "version",
    ],
    columns="item",
    values="response2",
    aggfunc="last",
).reset_index()

# Joining Wide format table with answers to include concatinated answer_ids
dat_wide = pd.merge(
    dat_wide,
    answers,
    on=[
        "userId",
        "secret_user_id",
        "activity_flow",
        "activity_flow_name",
        "event_id_report",
        "is_activity",
        "start_Time",
        "end_Time",
        "schedule_Time",
        "offset",
        "version",
    ],
    how="outer",
)

dat_wide.head()

### Specific item cleaning

In [ ]:
# Using the function from the Set Up section to cleanup time range and split start and end dates from the response

# Breaking up the old headache_time range question into 2
dat_wide[["headache_time_start_old", "headache_time_end_old"]] = dat_wide[
    "headache_time"
].str.split("/", expand=True)
dat_wide["headache_time_start_old"] = clean_time_range(
    dat_wide, "headache_time_start_old"
)
dat_wide["headache_time_end_old"] = clean_time_range(dat_wide, "headache_time_end_old")


# Cleanup gps response and split to 2 columns for lat and lon
dat_wide["now_gps"] = dat_wide["now_gps"].replace(r"[a-zA-Z\s+(\)]", "", regex=True)
dat_wide[["now_gps_lat", "now_gps_long"]] = dat_wide["now_gps"].str.split(
    "/", expand=True
)

#### Time Range Items

In [ ]:
# list of time range columns that need to be  split and formatted
# PLEASE ADD MORE TO THIS LIST IF NECESSARY
time_range_split = [
    "since_activity_monitor_time1",
    "since_activity_monitor_time",
    "since_light_device_time1",
    "since_light_device_time",
]

In [ ]:
for i in time_range_split:
    start = i + "_start"
    end = i + "_end"
    dat_wide[[start, end]] = dat_wide[i].str.split("/", expand=True)
    dat_wide[start] = clean_time_range(dat_wide, start)

In [ ]:
# Cleanup similar to R code
dat_wide_full = dat_wide.applymap(str)

### Timestamp cleaning

In [ ]:
# Epoch to Timestamp
dat_wide_full["start_Time"] = pd.to_numeric(
    dat_wide_full["start_Time"], errors="coerce"
)
dat_wide_full["start_Time"] = pd.to_datetime(
    dat_wide_full["start_Time"] / 1000, unit="s"
)

dat_wide_full["end_Time"] = pd.to_numeric(dat_wide_full["end_Time"], errors="coerce")
dat_wide_full["end_Time"] = pd.to_datetime(dat_wide_full["end_Time"] / 1000, unit="s")

dat_wide_full["schedule_Time"] = pd.to_numeric(
    dat_wide_full["schedule_Time"], errors="coerce"
)
dat_wide_full["schedule_Time"] = pd.to_datetime(
    dat_wide_full["schedule_Time"] / 1000, unit="s"
)

# Timestamp cleanup
dat_wide_full["schedule_Time"] = pd.to_datetime(dat_wide_full["schedule_Time"])
dat_wide_full["schedule_Time"] = np.where(
    dat_wide_full["offset"] == "1",
    dat_wide_full["schedule_Time"],
    dat_wide_full["schedule_Time"]
    .dt.tz_localize("UTC")
    .dt.tz_convert("America/New_York")
    .dt.tz_localize(None),
)

dat_wide_full["start_Time"] = pd.to_datetime(dat_wide_full["end_Time"])
dat_wide_full["start_Time"] = dat_wide_full["end_Time"].dt.floor("1s")
dat_wide_full["start_Time"] = np.where(
    dat_wide_full["offset"] == "1",
    dat_wide_full["end_Time"],
    dat_wide_full["start_Time"]
    .dt.tz_localize("UTC")
    .dt.tz_convert("America/New_York")
    .dt.tz_localize(None),
)

dat_wide_full["end_Time"] = pd.to_datetime(dat_wide_full["end_Time"])
dat_wide_full["end_Time"] = dat_wide_full["end_Time"].dt.floor("1s")
dat_wide_full["end_Time"] = np.where(
    dat_wide_full["offset"] == "1",
    dat_wide_full["end_Time"],
    dat_wide_full["end_Time"]
    .dt.tz_localize("UTC")
    .dt.tz_convert("America/New_York")
    .dt.tz_localize(None),
)

# Creating Dates
dat_wide_full["scheduled_Date"] = dat_wide_full["schedule_Time"].dt.date
dat_wide_full["start_Date"] = dat_wide_full["start_Time"].dt.date

In [ ]:
# Getting list of secret_user_id for userIds. This step could be excluded in future, once export file and schedule file have same secret_user_id
ID_List = dat_wide_full[["userId", "secret_user_id"]].drop_duplicates()

### Split data: Flows and activities

In [ ]:
if act_history_exist == 1:
    # Separting the data by flows vs activities
    act_dat_wide = dat_wide_full[dat_wide_full["is_activity"] == "Y"]
    dat_wide_full = dat_wide_full[dat_wide_full["is_activity"] == "N"]

    # Split activity_flow column to grab the activity_id and activity_name
    act_dat_wide[["activity_id_report", "activity_name_report"]] = act_dat_wide[
        "activity_flow"
    ].str.split("|", expand=True)

    # Dropping unnecessary columns
    act_dat_wide = act_dat_wide.drop(columns=["activity_flow", "is_activity"])

# Flow Submissions

### Flow schedule history cleaning

In [ ]:
# Timestamp formatting
history["scheduled_Time_start"] = (
    history["scheduled_date"] + " " + history["schedule_start_time"]
)
history["scheduled_Time_start"] = pd.to_datetime(history["scheduled_Time_start"])
history["scheduled_Date2"] = history["scheduled_Time_start"].dt.date

history["scheduled_Time_end"] = (
    history["scheduled_date"] + " " + history["schedule_end_time"]
)
history["scheduled_Time_end"] = pd.to_datetime(history["scheduled_Time_end"])

# Renaming columns
history = history.rename(columns={"secret_user_id": "history_sui"})

### Flow submission (report.csv) & Flow schedule history

In [ ]:
# Combine history table data with wide data to get missing schedule rows
conn = sqlite3.connect(":memory:")  # Make the db in memory
# write the tables
history.to_sql("history", conn, index=False)
dat_wide_full.to_sql("dat_wide_full", conn, index=False)

qry = """
    select  
        *
    from
        history full outer join dat_wide_full
            on history.user_id = dat_wide_full.userId
            and history.event_id = dat_wide_full.event_id_report
            and history.flow_id = dat_wide_full.activity_flow
            and strftime('%Y-%m-%d %H:%M:%S' , dat_wide_full.schedule_Time) between strftime('%Y-%m-%d %H:%M:%S' , history.scheduled_Time_start) and strftime('%Y-%m-%d %H:%M:%S', history.scheduled_Time_end) 
    """
dat_joined = pd.read_sql_query(qry, conn)

Check if joined correctly

In [ ]:
# Check if joined correctly
## !! Should not be 0 !!

len(
    dat_joined[
        (pd.notna(dat_joined["history_sui"])) & (pd.notna(dat_joined["userId"]))
    ].index
)

In [ ]:
# Export and Check joined data, if necessary

# dat_joined.to_csv(os.path.join(output_path, 'flow_joined_check.csv'), index=False, date_format='%Y-%m-%d %H:%M:%S', na_rep='NA')

### Flow Items cleaning

In [ ]:
if act_history_exist == 0:
    # Setting this for binary values
    flow["value"] = 1
    flow.rename(columns={"applet_version": "version"}, inplace=True)
    # Wide binary column build
    flow_wide = pd.pivot_table(
        flow,
        index=["version", "flow_id"],
        columns="activity_name",
        values="value",
        aggfunc="first",
    )
    flow_wide = flow_wide.fillna(0)
    flow_wide = flow_wide.reset_index()
else:
    # Creating activities binary columns
    flow_activities_report = act_dat_wide.copy()
    flow_activities_history = act_history.copy()

    flow_activities_report = flow_activities_report[
        ["activity_id_report", "activity_name_report", "version"]
    ]
    flow_activities_history = flow_activities_history[
        ["activity_id", "activity_name", "applet_version"]
    ]

    flow_activities_report.rename(
        columns={
            "activity_id_report": "activity_id",
            "activity_name_report": "activity_name",
            "version": "applet_version",
        },
        inplace=True,
    )

    flow_activities_report["value"] = 0
    flow_activities_history["value"] = 0

    flow_activities_all = [flow_activities_report, flow_activities_history]
    flow_activities = pd.concat(flow_activities_all, ignore_index=True)

    # Dropping duplicates for join
    flow_activities = flow_activities.drop_duplicates()

    # Getting other binary columns from flow history to ensure all activities are listed
    flows = flow.copy()
    flows = flows[["flow_id", "activity_name", "applet_version"]]
    flows.rename(columns={"applet_version": "version"}, inplace=True)
    flows["value_flow"] = 1

    # Dropping duplicates for join
    flows = flows.drop_duplicates()

    # Joining the two binary column data together
    flows_final = pd.merge(
        flows,
        flow_activities,
        how="outer",
        left_on=["version", "flow_id"],
        right_on=["applet_version", "activity_id"],
    )

    # Filling in data with the join so all information is in one column
    flows_final["flow_id_dup"] = flows_final["flow_id"]
    flows_final["flow_id"] = np.where(
        flows_final["flow_id_dup"].isna(),
        flows_final["activity_id"],
        flows_final["flow_id_dup"],
    )

    flows_final["activity_name"] = flows_final["activity_name_x"]
    flows_final["activity_name"] = np.where(
        flows_final["activity_name_x"].isna(),
        flows_final["activity_name_y"],
        flows_final["activity_name_x"],
    )

    flows_final["version_dup"] = flows_final["version"]
    flows_final["version"] = np.where(
        flows_final["version_dup"].isna(),
        flows_final["applet_version"],
        flows_final["version_dup"],
    )

    flows_final["value_dup"] = flows_final["value"]
    flows_final["value"] = np.where(
        flows_final["value_flow"].isna(),
        flows_final["value_dup"],
        flows_final["value_flow"],
    )

    flows_final = flows_final[["flow_id", "activity_name", "version", "value"]]

    # Making binary data wide to join and filling any NAs with 0
    flow_wide = pd.pivot_table(
        flows_final,
        index=["version", "flow_id"],
        columns="activity_name",
        values="value",
        aggfunc="first",
    ).reset_index()
    flow_wide = flow_wide.fillna(0)

### Joined Flows Data and Flow Items

In [ ]:
# Getting the version column such that binary values exist for null response rows
dat_joined["version_report"] = dat_joined["version"]
dat_joined["version"] = np.where(
    dat_joined["version_report"].isna(),
    dat_joined["applet_version"],
    dat_joined["version_report"],
)

# Getting the flow column such that binary values exist for null response rows
dat_joined["activity_flow_id"] = np.where(
    dat_joined["activity_flow"].isna(),
    dat_joined["flow_id"],
    dat_joined["activity_flow"],
)
dat_joined["activity_flow"] = np.where(
    dat_joined["activity_flow_name"].isna(),
    dat_joined["flow_name"],
    dat_joined["activity_flow_name"],
)

# Merging Wide fomart export data with binary wide data
dat_wide2 = pd.merge(
    dat_joined,
    flow_wide,
    how="left",
    left_on=["version", "activity_flow_id"],
    right_on=["version", "flow_id"],
)

# Getting the user_id column to ensure user_id is always available
dat_wide2["userId_report"] = dat_wide2["userId"]
dat_wide2["userId"] = np.where(
    dat_wide2["userId_report"].isna(), dat_wide2["user_id"], dat_wide2["userId_report"]
)

# Combining Headache Time separated version and time range version together
dat_wide2["headache_time_start_dup"] = dat_wide2["headache_time_start"]
dat_wide2["headache_time_start"] = np.where(
    (dat_wide2["headache_time_start_dup"] == "nan")
    | (dat_wide2["headache_time_start_dup"].isna()),
    dat_wide2["headache_time_start_old"],
    dat_wide2["headache_time_start_dup"],
)
dat_wide2["headache_time_end_dup"] = dat_wide2["headache_time_end"]
dat_wide2["headache_time_end"] = np.where(
    (dat_wide2["headache_time_end_dup"] == "nan")
    | (dat_wide2["headache_time_end_dup"].isna()),
    dat_wide2["headache_time_end_old"],
    dat_wide2["headache_time_end_dup"],
)

# Timestamp formatting - TO MAKE SURE!
dat_wide2["schedule_Time"] = pd.to_datetime(dat_wide2["schedule_Time"])
dat_wide2["start_Time"] = pd.to_datetime(dat_wide2["start_Time"])
dat_wide2["end_Time"] = pd.to_datetime(dat_wide2["end_Time"])
dat_wide2["scheduled_Time_start"] = pd.to_datetime(dat_wide2["scheduled_Time_start"])

# Getting secret_user_id for all the user_ids. This step could be excluded in future, once export file and schedule file have same secret_user_id
final = pd.merge(
    dat_wide2, ID_List, how="left", left_on=["userId"], right_on=["userId"]
)

### Flows Final

In [ ]:
#  Ensure secret_user_id available for all rows
final["secret_user_id"] = np.where(
    final["secret_user_id_x"].isna(),
    final["secret_user_id_y"],
    final["secret_user_id_x"],
)
final["secret_user_id"] = np.where(
    final["secret_user_id"].isna(), final["history_sui"], final["secret_user_id"]
)

# If activity schedule time is null, it will be replaced with timestamp from schedule file
final["schedule_Time"] = np.where(
    final["schedule_Time"].isna(), final["scheduled_Time_start"], final["schedule_Time"]
)

# If event_id is null, it will be replaced with event_id from schedule file
final["event_id_sched"] = final["event_id"]
final["event_id"] = np.where(
    final["event_id_report"].isna(), final["event_id_sched"], final["event_id_report"]
)

In [ ]:
#################################################################### PLEASE SELECT APPROPRIATE ITEMS & ORDER ###############################################################

# selecting the required columns and ordering

final = final[
    [
        "userId",
        "event_id",
        "id",
        "secret_user_id",
        "activity_flow",
        "schedule_Time",
        "start_Time",
        "end_Time",
        "version",
        " Sleep ",
        "Activity Watch",
        "Context of Assessment",
        "Diet",
        #  'Diet Test Activity schedule', #TESTING ACTIVITY
        "Food and Drink Intake",
        "Food and Drink Intake (Supplemental)",  # Only in Testing
        "Life Events",
        "Light Device",
        "Menstrual Period",
        "Mood Circumplex",
        "Pain",
        "Physical Activity",
        "Physical Activity (with Supplement)",  # Only in Testing
        "Physical Health",
        "Procedure Tracking",  # Only in Testing
        "Saliva Sample",
        "Substance Use",  # Only in Testing
        "Substance Use (AM)",  # Only in Testing
        "Substance Use (Core)",  # Only in Testing
        "Substance Use (Supplement)",
        "saliva_instructions",
        "since_activity_monitor",
        "since_activity_monitor_time_start",
        "since_activity_monitor_time_end",
        "since_light_device",
        "since_light_device_time_start",
        "since_light_device_time_end",
        "morning_bedtime",
        "morning_lights_off",
        "morning_fall_asleep",
        "morning_wake_number",
        "morning_awakenings_last",
        "morning_waketime",
        "morning_outbed",
        "morning_sleep_quality",
        "morning_sleep_refreshed",
        "morning_sleep_problems",
        "morning_sleeping_pills",
        "morning_sleeping_pills_type",
        "now_gps_lat",
        "now_gps_long",
        "now_where",
        "now_inside",
        "now_outside",
        "since_where",
        "since_inside",
        "since_outside",
        "now_company",
        # 'now_company_confirm',
        "since_company",
        "now_activity",
        "since_activity",
        "since_internet",
        "internet_use_duration",
        "internet_use_category",
        "videogame_duration",
        "socialmedia_duration",
        "socialmedia_activity",
        "friends_communication_method",
        "strangers_communication_method",
        "comments_byothers",
        "comments_byself",
        "now_sadness",
        "now_anxiousness",
        "now_active",
        "now_tired",
        "now_distracted",
        "now_irritable",
        "now_quick_thinking",
        "now_enjoyment",
        "now_fidgety",
        "now_thoughts_positive",
        "now_thoughts_negative",
        "now_thoughts_negative_about",
        "now_thoughts_negative_severity",
        "since_thoughts_negative",
        "since_thoughts_negative_suicide",
        "since_thoughts_negative_suicide_message",
        "instructions",  # event_instructions
        "event_emotion",
        "event_category",
        "event_people",
        "event_where",
        "event_health",
        "event_content",
        "event_issue",
        "event_stress",
        "since_physical_activity",
        "since_sedentary",
        "since_nap_rest_time",
        "since_rest_duration",
        "since_rest_fell_asleep",
        "since_sleep_duration",
        "since_vigorous_activity",
        "since_moderate_activity",
        "since_light_activity",
        "now_thirsty",
        "since_had_drink",
        "not_alcohol_amount",
        "since_had_drink_alcohol_type",
        "since_had_drink_alcohol_quantity",
        "alcohol_time",
        "since_feel_drink",
        "since_had_drink_caffeinated_type",
        "now_hungry",
        "since_times_eat",
        "since_eaten_amount",
        "since_eaten_when",
        "since_eaten_type",
        "since_food1",
        "since_food2",
        "since_food3",
        "since_food4",
        "since_food5",
        "since_crave",
        "craving_strong_tobacco",
        "craving_strong_cannabis",
        "craving_strong_otherdrug",
        "craving_strong_alcohol",
        "since_substances",
        "substances_tobacco",
        "since_substances_cigarettes",
        "since_substances_cigarettes_time",
        "since_substances_enicotine",
        "since_substances_enicotine_time",
        "since_substances_other_tobacco",
        "since_substances_other_tobacco_time",
        "since_cannabis_type",
        "since_cannabis_time",
        "since_substances_cannabis",
        "since_cannabis_high",
        "since_substances_other",
        "since_substances_other_specify",
        "since_substances_other_time",
        "since_substances_other_high",
        "since_substances_company",
        "now_pain",
        "now_pain_where",
        "since_pain",
        "since_pain_where",
        "pain_intensity",
        "headache",
        "headache_prevent",
        "headache_time_start",
        "headache_time_end",
        "headache_intensity",
        "headache_location",
        "headache_pulsating",
        "headache_effort",
        "headache_nausea",
        "headache_light",
        "headache_heat",
        "headache_noise",
        "headache_smell",
        "headache_trigger",
        "headache_vision_changes",
        "headache_vision_change_time",
        "headache_numbing",
        "headache_numbing_time",
        "headache_confusing",
        "headache_confusing_time",
        "headache_sudden",
        "headache_medication",
        "headache_interference",
        "day_physical_health",
        "day_problem_categories",
        "day_bother",
        "day_over_medication",
        "day_over_medication_why",
        "day_prescribed_medication",
        "day_prescribed_medication_conditions",
        "day_problems_allergies",
        "day_problems_breath",
        "day_problems_belly_symptoms",
        "day_problems_belly",
        "day_problems_muscle",
        "day_problems_heart",
        "dizziness_situation",
        "dizziness_faint",
        "day_lethargic",
        "activity_planned",
        "day_period",
        ##New items (2024 Jul 31st)
        "audio_test",
        "headache_current",
        "headache_same",
        "since_activity_monitor_now",
        # 'since_cannabis_craving',
        # 'since_light_activity_planned',
        "since_light_device_now",
        # 'since_moderate_activity_planned',
        # 'since_substances_other_crave',
        # 'since_vigorous_activity_planned',
        # 'substances_lastnight_time',
        # 'substances_lastnight'
    ]
]

In [ ]:
# Ensuring consistent NA wording across all columns
final_out = final.copy()
final_out.replace("nan", "NA", inplace=True)
final_out.fillna("NA", inplace=True)
final_out.replace("NA NA", "NA", inplace=True)
final_out.rename(columns={"userId_x": "user_id"}, inplace=True)

# Excluding test flows
final_out = final_out[final_out["activity_flow"] != "Test Flow (All Activities)"]

# Checking final flow data
final_out.head()

In [ ]:
# write out final flow dataframe, if necessary:

final_out.to_csv(
    os.path.join(output_path, "flow_final.csv"),
    index=False,
    date_format="%Y-%m-%d %H:%M:%S",
    na_rep="NA",
)

# Activity Submissions

In [ ]:
# Check ACTIVITIES Submissions data:
# act_dat_wide.head()

### Activity schedule history cleaning

In [ ]:
if act_history_exist == 1:
    # Timestamp formatting
    act_history["scheduled_Time_start"] = (
        act_history["scheduled_date"] + " " + act_history["schedule_start_time"]
    )
    act_history["scheduled_Time_start"] = pd.to_datetime(
        act_history["scheduled_Time_start"]
    )
    act_history["scheduled_Date2"] = act_history["scheduled_Time_start"].dt.date

    act_history["scheduled_Time_end"] = (
        act_history["scheduled_date"] + " " + act_history["schedule_end_time"]
    )
    act_history["scheduled_Time_end"] = pd.to_datetime(
        act_history["scheduled_Time_end"]
    )

    # Renaming columns
    act_history = act_history.rename(columns={"secret_user_id": "history_sui"})

### Activity Submission (report.csv) & Activity schedule history

In [ ]:
if act_history_exist == 1:
    # Combine history table data with wide data to get missing schedule rows
    conn = sqlite3.connect(":memory:")  # Make the db in memory
    # write the tables
    act_history.to_sql("history", conn, index=False)
    act_dat_wide.to_sql("dat_wide_full", conn, index=False)

    qry = """
        select  
            *
        from
            history full outer join dat_wide_full
            on history.user_id = dat_wide_full.userId
            and history.event_id = dat_wide_full.event_id_report
            and history.activity_id = dat_wide_full.activity_id_report
            and strftime('%Y-%m-%d %H:%M:%S' , dat_wide_full.schedule_Time) between strftime('%Y-%m-%d %H:%M:%S' , history.scheduled_Time_start) and strftime('%Y-%m-%d %H:%M:%S', history.scheduled_Time_end) 
        """
    act_dat_joined = pd.read_sql_query(qry, conn)

Check if it joined correctly:

In [ ]:
# Check if joined correctly
## !! Should not be 0 !!
if act_history_exist == 1:
    print(
        len(
            act_dat_joined[
                (pd.notna(act_dat_joined["history_sui"]))
                & (pd.notna(act_dat_joined["userId"]))
            ].index
        )
    )

In [ ]:
# Export and Check joined data, if necessary

# if act_history_exist == 1:
#     dat_joined.to_csv(os.path.join(output_path, 'activities_joined_check.csv'), index=False, date_format='%Y-%m-%d %H:%M:%S', na_rep='NA')

### Joined Data Cleaning

In [ ]:
if act_history_exist == 1:
    # Getting the user_id column to ensure user_id is always available
    act_dat_joined["userId_report"] = act_dat_joined["userId"]
    act_dat_joined["userId"] = np.where(
        act_dat_joined["userId_report"].isna(),
        act_dat_joined["user_id"],
        act_dat_joined["userId_report"],
    )

    # Getting the version column such that binary values exist for null response rows
    act_dat_joined["version_report"] = act_dat_joined["version"]
    act_dat_joined["version"] = np.where(
        act_dat_joined["version_report"].isna(),
        act_dat_joined["applet_version"],
        act_dat_joined["version_report"],
    )

    # If event_id is null, it will be replaced with event_id from schedule file
    act_dat_joined["event_id_sched"] = act_dat_joined["event_id"]
    act_dat_joined["event_id"] = np.where(
        act_dat_joined["event_id_report"].isna(),
        act_dat_joined["event_id_sched"],
        act_dat_joined["event_id_report"],
    )

    #  Ensure secret_user_id available for all rows
    act_dat_joined["secret_user_id"] = np.where(
        act_dat_joined["secret_user_id"].isna(),
        act_dat_joined["history_sui"],
        act_dat_joined["secret_user_id"],
    )

    #  Ensure activity_name available for all rows
    act_dat_joined["activity_name_history"] = act_dat_joined["activity_name"]
    act_dat_joined["activity_name"] = np.where(
        act_dat_joined["activity_name_report"].isna(),
        act_dat_joined["activity_name_history"],
        act_dat_joined["activity_name_report"],
    )

    #  Ensure activity_id available for all rows
    act_dat_joined["activity_id_history"] = act_dat_joined["activity_id"]
    act_dat_joined["activity_id"] = np.where(
        act_dat_joined["activity_id_report"].isna(),
        act_dat_joined["activity_id_history"],
        act_dat_joined["activity_id_report"],
    )

    # Combining Headache Time separated version and time range version together
    act_dat_joined["headache_time_start_dup"] = act_dat_joined["headache_time_start"]
    act_dat_joined["headache_time_start"] = np.where(
        (act_dat_joined["headache_time_start_dup"] == "nan")
        | (act_dat_joined["headache_time_start_dup"].isna()),
        act_dat_joined["headache_time_start_old"],
        act_dat_joined["headache_time_start_dup"],
    )
    act_dat_joined["headache_time_end_dup"] = act_dat_joined["headache_time_end"]
    act_dat_joined["headache_time_end"] = np.where(
        (act_dat_joined["headache_time_end_dup"] == "nan")
        | (act_dat_joined["headache_time_end_dup"].isna()),
        act_dat_joined["headache_time_end_old"],
        act_dat_joined["headache_time_end_dup"],
    )

    # Timestamp formatting - TO MAKE SURE!
    act_dat_joined["schedule_Time"] = pd.to_datetime(act_dat_joined["schedule_Time"])
    act_dat_joined["start_Time"] = pd.to_datetime(act_dat_joined["start_Time"])
    act_dat_joined["end_Time"] = pd.to_datetime(act_dat_joined["end_Time"])
    act_dat_joined["scheduled_Time_start"] = pd.to_datetime(
        act_dat_joined["scheduled_Time_start"]
    )

### Activity Name Binary Column Data Creation

In [ ]:
if act_history_exist == 1:
    # Creating activities binary columns
    activities = act_dat_joined.copy()
    activities = activities[["activity_id", "activity_name", "version"]]
    activities["value"] = 1

    # Dropping duplicates for join
    activities = activities.drop_duplicates()

    # Getting other binary columns from flow history to ensure all activities are listed
    other_activities = flow.copy()
    other_activities = flow[["activity_id", "activity_name", "applet_version"]]
    other_activities["value_flow"] = 0

    # Dropping duplicates for join
    other_activities = other_activities.drop_duplicates()

    # Joining the two binary column data together
    activities_final = pd.merge(
        other_activities,
        activities,
        how="outer",
        left_on=["applet_version", "activity_id"],
        right_on=["version", "activity_id"],
    )

    # Filling in data with the join so all information is in one column
    activities_final["values"] = np.where(
        activities_final["value"].isna(),
        activities_final["value_flow"],
        activities_final["value"],
    )
    activities_final["activity_name"] = np.where(
        activities_final["activity_name_x"].isna(),
        activities_final["activity_name_y"],
        activities_final["activity_name_x"],
    )
    activities_final["version_activity"] = activities_final["version"]
    activities_final["version"] = np.where(
        activities_final["applet_version"].isna(),
        activities_final["version_activity"],
        activities_final["applet_version"],
    )
    activities_final = activities_final[
        ["activity_id", "activity_name", "version", "values"]
    ]

    # Making binary data wide to join and filling any NAs with 0
    activities_wide = pd.pivot_table(
        activities_final,
        index=[
            "version",
            "activity_id",
            "activity_name",
        ],
        columns="activity_name",
        values="values",
        aggfunc="first",
    ).reset_index()
    activities_wide = activities_wide.fillna(0)

### Joined Data and Activities Binary

In [ ]:
if act_history_exist == 1:
    # Merging Wide fomart export data with binary wide data
    act_final = pd.merge(
        act_dat_joined,
        activities_wide,
        how="left",
        left_on=["version", "activity_id"],
        right_on=["version", "activity_id"],
    )

    # Ensuring schedule time shows up
    act_final["schedule_Time"] = np.where(
        act_final["schedule_Time"].isna(),
        act_final["scheduled_Time_start"],
        act_final["schedule_Time"],
    )

    # Final Timestamp formatting
    act_final["schedule_Time"] = pd.to_datetime(act_final["schedule_Time"])
    act_final["start_Time"] = pd.to_datetime(act_final["start_Time"])
    act_final["end_Time"] = pd.to_datetime(act_final["end_Time"])

    # Creating activity_flow column to match the flow final data
    act_final["activity_flow"] = np.nan

    # Check to make sure you see all activity names
    list(activities_wide.columns)

### Activities Final

In [ ]:
################################################################################### PLEASE SELECT APPROPRIATE ITEMS & ORDER ############################################################################

# selecting the required columns and ordering
if act_history_exist == 1:
    act_final = act_final[
        [
            "userId",
            "event_id",
            "id",
            "secret_user_id",
            "activity_flow",
            "schedule_Time",
            "start_Time",
            "end_Time",
            "version",
            " Sleep ",
            "Activity Watch",
            "Context of Assessment",
            "Diet",
            "Diet Test Activity schedule",  # TESTING ACTIVITY
            "Food and Drink Intake",
            "Food and Drink Intake (Supplemental)",  # Only in Testing
            "Life Events",
            "Light Device",
            "Menstrual Period",
            "Mood Circumplex",
            "Pain",
            "Physical Activity",
            "Physical Activity (with Supplement)",  # Only in Testing
            "Physical Health",
            "Procedure Tracking",  # Only in Testing
            "Saliva Sample",
            "Substance Use",  # Only in Testing
            "Substance Use (AM)",  # Only in Testing
            "Substance Use (Core)",  # Only in Testing
            "Substance Use (Supplement)",
            "saliva_instructions",
            "since_activity_monitor",
            "since_activity_monitor_time_start",
            "since_activity_monitor_time_end",
            "since_light_device",
            "since_light_device_time_start",
            "since_light_device_time_end",
            "morning_bedtime",
            "morning_lights_off",
            "morning_fall_asleep",
            "morning_wake_number",
            "morning_awakenings_last",
            "morning_waketime",
            "morning_outbed",
            "morning_sleep_quality",
            "morning_sleep_refreshed",
            "morning_sleep_problems",
            "morning_sleeping_pills",
            "morning_sleeping_pills_type",
            "now_gps_lat",
            "now_gps_long",
            "now_where",
            "now_inside",
            "now_outside",
            "since_where",
            "since_inside",
            "since_outside",
            "now_company",
            # 'now_company_confirm',
            "since_company",
            "now_activity",
            "since_activity",
            "since_internet",
            "internet_use_duration",
            "internet_use_category",
            "videogame_duration",
            "socialmedia_duration",
            "socialmedia_activity",
            "friends_communication_method",
            "strangers_communication_method",
            "comments_byothers",
            "comments_byself",
            "now_sadness",
            "now_anxiousness",
            "now_active",
            "now_tired",
            "now_distracted",
            "now_irritable",
            "now_quick_thinking",
            "now_enjoyment",
            "now_fidgety",
            "now_thoughts_positive",
            "now_thoughts_negative",
            "now_thoughts_negative_about",
            "now_thoughts_negative_severity",
            "since_thoughts_negative",
            "since_thoughts_negative_suicide",
            "since_thoughts_negative_suicide_message",
            "instructions",  # event_instructions
            "event_emotion",
            "event_category",
            "event_people",
            "event_where",
            "event_health",
            "event_content",
            "event_issue",
            "event_stress",
            "since_physical_activity",
            "since_sedentary",
            "since_nap_rest_time",
            "since_rest_duration",
            "since_rest_fell_asleep",
            "since_sleep_duration",
            "since_vigorous_activity",
            "since_moderate_activity",
            "since_light_activity",
            "now_thirsty",
            "since_had_drink",
            "not_alcohol_amount",
            "since_had_drink_alcohol_type",
            "since_had_drink_alcohol_quantity",
            "alcohol_time",
            "since_feel_drink",
            "since_had_drink_caffeinated_type",
            "now_hungry",
            "since_times_eat",
            "since_eaten_amount",
            "since_eaten_when",
            "since_eaten_type",
            "since_food1",
            "since_food2",
            "since_food3",
            "since_food4",
            "since_food5",
            "since_crave",
            "craving_strong_tobacco",
            "craving_strong_cannabis",
            "craving_strong_otherdrug",
            "craving_strong_alcohol",
            "since_substances",
            "substances_tobacco",
            "since_substances_cigarettes",
            "since_substances_cigarettes_time",
            "since_substances_enicotine",
            "since_substances_enicotine_time",
            "since_substances_other_tobacco",
            "since_substances_other_tobacco_time",
            "since_cannabis_type",
            "since_cannabis_time",
            "since_substances_cannabis",
            "since_cannabis_high",
            "since_substances_other",
            "since_substances_other_specify",
            "since_substances_other_time",
            "since_substances_other_high",
            "since_substances_company",
            "now_pain",
            "now_pain_where",
            "since_pain",
            "since_pain_where",
            "pain_intensity",
            "headache",
            "headache_prevent",
            "headache_time_start",
            "headache_time_end",
            "headache_intensity",
            "headache_location",
            "headache_pulsating",
            "headache_effort",
            "headache_nausea",
            "headache_light",
            "headache_heat",
            "headache_noise",
            "headache_smell",
            "headache_trigger",
            "headache_vision_changes",
            "headache_vision_change_time",
            "headache_numbing",
            "headache_numbing_time",
            "headache_confusing",
            "headache_confusing_time",
            "headache_sudden",
            "headache_medication",
            "headache_interference",
            "day_physical_health",
            "day_problem_categories",
            "day_bother",
            "day_over_medication",
            "day_over_medication_why",
            "day_prescribed_medication",
            "day_prescribed_medication_conditions",
            "day_problems_allergies",
            "day_problems_breath",
            "day_problems_belly_symptoms",
            "day_problems_belly",
            "day_problems_muscle",
            "day_problems_heart",
            "dizziness_situation",
            "dizziness_faint",
            "day_lethargic",
            "activity_planned",
            "day_period",
            ##New items (2024 Jul 31st)
            "audio_test",
            "headache_current",
            "headache_same",
            "since_activity_monitor_now",
            # 'since_cannabis_craving',
            # 'since_light_activity_planned',
            "since_light_device_now",
            # 'since_moderate_activity_planned',
            # 'since_substances_other_crave',
            # 'since_vigorous_activity_planned',
            # 'substances_lastnight_time',
            # 'substances_lastnight'
        ]
    ]

In [ ]:
if act_history_exist == 1:
    # Ensuring all NAs apper "NA"
    act_final_out = act_final.copy()
    act_final_out.replace("nan", "NA", inplace=True)
    act_final_out.fillna("NA", inplace=True)
    act_final_out.replace("NA NA", "NA", inplace=True)

In [ ]:
# Checking to see activities final output looks good.
if act_history_exist == 1:
    act_final_out.to_csv(
        os.path.join(output_path, "activity_final.csv"),
        index=False,
        date_format="%Y-%m-%d %H:%M:%S",
        na_rep="NA",
    )

# Final Output

In [ ]:
if act_history_exist == 1:
    # Joining flow flow and activity final togeter
    all_final = [final_out, act_final_out]
    ema_out = pd.concat(all_final, ignore_index=True)

    # Included similar to R script to ensure alignment with formatting
    ema_out = ema_out.applymap(str)

    # Final data output sorting
    ema_out = ema_out.sort_values(by=["secret_user_id", "schedule_Time"])

Check final output

In [ ]:
if act_history_exist == 1:
    # checking for duplicates in final file. Ensure this always returns "False"
    print(ema_out.duplicated().any())

In [ ]:
if act_history_exist == 1:
    # Preview final output
    ema_out.head()

In [ ]:
if act_history_exist == 1:
    # Write the final output file to csv

    ema_out.to_csv(
        os.path.join(output_path, "ema_output_final_v2.csv"),
        index=False,
        date_format="%Y-%m-%d %H:%M:%S",
        na_rep="NA",
    )